In [2]:
import numpy as np
from scipy.stats import norm

def cumulative_sums_test(sequence: str, mode: int):
    """
    Executa o Cumulative Sums (Cusum) Test do NIST SP800-22.

    Parâmetros:
    - sequence (str): sequência binária (ex: "101001...")
    - mode (int): 0 para forward, 1 para backward

    Retorna:
    - z (float): maior excursão da caminhada aleatória
    - p_value (float): valor-p do teste
    """
    n = len(sequence)

    # Etapa 1: converter bits para -1 e +1
    x = np.array([2 * int(bit) - 1 for bit in sequence], dtype=int)

    # Etapa 2: calcular a soma cumulativa (modo direto ou reverso)
    if mode == 0:
        s = np.cumsum(x)
    else:
        s = np.cumsum(x[::-1])

    # Etapa 3: maior excursão absoluta
    z = np.max(np.abs(s))

    # Etapa 4: cálculo do valor-p com base na distribuição normal padrão
    terms = []
    start = int((-n / z + 1) / 4)
    end = int((n / z - 1) / 4) + 1
    for k in range(start, end + 1):
        term = norm.cdf((4 * k + 1) * z / np.sqrt(n)) - norm.cdf((4 * k - 1) * z / np.sqrt(n))
        terms.append(term)

    sum1 = np.sum(terms)

    terms = []
    start = int((-n / z - 3) / 4)
    end = int((n / z - 1) / 4) + 1
    for k in range(start, end + 1):
        term = norm.cdf((4 * k + 3) * z / np.sqrt(n)) - norm.cdf((4 * k + 1) * z / np.sqrt(n))
        terms.append(term)

    sum2 = np.sum(terms)

    p_value = 1.0 - sum1 + sum2

    return z, p_value


# ===============================
# 🧪 EXEMPLO NIST (Seção 2.14.8)
# ===============================
if __name__ == "__main__":
    sequence = (
        "11001001000011111101101010100010001000010110100011"
        "00001000110100110001001100011001100010100010111000"
    )

    print("==== Cumulative Sums Test ====")
    for mode in [0, 1]:
        z, p_val = cumulative_sums_test(sequence, mode)
        modo = "Forward" if mode == 0 else "Reverse"
        print(f"\nModo: {modo}")
        print(f"z (maior excursão): {z:.6f}")
        print(f"P-valor: {p_val:.6f}")
        print("Conclusão:", "✅ RANDOM" if p_val >= 0.01 else "❌ NÃO ALEATÓRIA")


==== Cumulative Sums Test ====

Modo: Forward
z (maior excursão): 16.000000
P-valor: 0.219194
Conclusão: ✅ RANDOM

Modo: Reverse
z (maior excursão): 19.000000
P-valor: 0.114866
Conclusão: ✅ RANDOM
